In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
traintanic = pd.read_csv('train.csv')
testanic = pd.read_csv('test.csv')

Features:
Ticket class, sex, age, parent-child, sibling-spouse, Tune: nationality (split off last names and classify non-Anglos by name ending)

Start with decision tree, then do KNN, then SVM
See what happens when combined

To do: separate out deck from cabin number

Assign deck as well as fore, amidships or aft cabin numbers based on ticket class

Check on how exactly the ship sank - 

Create column for young children (try different limit downwards from 16)

Ignore ticket & fare

Ignore port of embarkation


'Foreign' name endings:
off
ov
cher
sky
ski
ig
nen
sen
ik
strom
ic
sen
blom
lund
og

In [7]:
traintanic['Last_Name'] = traintanic['Name'].str.split(',', n = 1, expand = True)

In [8]:
traintanic['Is Non-Anglo Western?'] = traintanic.Name.apply\
(lambda x: 1 if re.compile('adt,|berg,|berger,|blom,|burg,|burger,|a,|o,|elle,|elli,|ehr,|ig,|ila,|cher,\
|nen,|sen,|ik,|ek,|og,|our,|gen,|lund,|strom,|sson,|sen,|vedt,|stad,|stedt,| van |qvist,|quist,\
            |aki,|feld,|mann,|gli,|glin,|ahl,|ouf,| Van| Vander').search(x) else 0)

testanic['Is Non-Anglo Western?'] = testanic.Name.apply\
(lambda x: 1 if re.compile('adt,|berg,|berger,|blom,|burg,|burger,|a,|o,|elle,|elli,|ehr,|ig,|ila,|cher,\
|nen,|sen,|ik,|ek,|og,|our,|gen,|lund,|strom,|sson,|sen,|vedt,|stad,|stedt,| van |qvist,|quist,\
            |aki,|feld,|mann,|gli,|glin,|ahl,|ouf,| Van| Vander').search(x) else 0)

In [9]:
traintanic['Is Eastern?'] = traintanic.Name.apply\
(lambda x: 1 if re.compile('ac,|off,|eff,|if,|ov,|ic,|sky,|ski,|u,|ib,|os,|oub,').search(x) else 0)

testanic['Is Eastern?'] = testanic.Name.apply\
(lambda x: 1 if re.compile('ac,|off,|eff,|if,|ov,|ic,|sky,|ski,|u,|ib,|os,|oub,').search(x) else 0)

In [10]:
traintanic['Is 1st Class?'] = traintanic.Pclass.apply(lambda x: 1 if x == 1 else 0)

testanic['Is 1st Class?'] = testanic.Pclass.apply(lambda x: 1 if x == 1 else 0)

In [11]:
traintanic['Is 3rd Class?'] = traintanic.Pclass.apply(lambda x: 1 if x == 3 else 0)

testanic['Is 3rd Class?'] = testanic.Pclass.apply(lambda x: 1 if x == 3 else 0)

In [12]:
traintanic['Deck'] = traintanic.Cabin.str.get(0)
traintanic['Cabin'] = traintanic.Cabin.astype('object')

testanic['Deck'] = testanic.Cabin.str.get(0)
testanic['Cabin'] = testanic.Cabin.astype('object')

In [13]:
traintanic['Cabin_No'] = traintanic['Cabin'].str[1:4]

testanic['Cabin_No'] = testanic['Cabin'].str[1:4]

In [14]:
traintanic.Deck = traintanic.Deck.fillna(0)
traintanic.Cabin_No = traintanic.Cabin_No.fillna(0)

testanic.Deck = testanic.Deck.fillna(0)
testanic.Cabin_No = testanic.Cabin_No.fillna(0)

In [15]:
traintanic['Is Child?'] = traintanic.Age.apply(lambda x: 1 if x <= 12 else 0)
traintanic['Is Young Adult?'] = traintanic.Age.apply(lambda x: 1 if x in range(13,31) else 0)

testanic['Is Child?'] = testanic.Age.apply(lambda x: 1 if x <= 12 else 0)
testanic['Is Young Adult?'] = testanic.Age.apply(lambda x: 1 if x in range(13,31) else 0)

In [16]:
traintanic['Is Not Young?'] = traintanic.Age.apply(lambda x: 1 if x > 40 else 0)

testanic['Is Not Young?'] = testanic.Age.apply(lambda x: 1 if x > 40 else 0)

In [17]:
traintanic['Big Family?'] = traintanic.Parch.apply(lambda x: 1 if x > 2 else 0)

testanic['Big Family?'] = testanic.Parch.apply(lambda x: 1 if x > 2 else 0)

In [18]:
traintanic[['Is Child?','Is Young Adult?']].fillna(0, inplace=True)

testanic[['Is Child?','Is Young Adult?']].fillna(0, inplace=True)

C:\Users\teren\miniconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [19]:
traintanic.Cabin_No.replace(to_replace=' G7', value=7, inplace=True)
traintanic.Cabin_No.replace(to_replace=' G6', value=6, inplace=True)
traintanic.Cabin_No.replace(to_replace=' E6', value=6, inplace=True)
traintanic.Cabin_No.replace(to_replace='', value=0, inplace=True)

In [20]:
testanic.Cabin_No.replace(to_replace=' [A-Z]', value='', inplace=True, regex=True)
testanic.Cabin_No.replace(to_replace='', value='0', inplace=True, regex=True)

In [21]:
traintanic['Cabin_No'] = traintanic['Cabin_No'].astype('int64')

testanic['Cabin_No'] = testanic['Cabin_No'].astype('int64')

In [22]:
traintanic['Cabin Fore?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x in range(66) else 0)

testanic['Cabin Fore?'] = testanic['Cabin_No'].apply(lambda x: 1 if x in range(66) else 0)

In [23]:
traintanic['Cabin Aft?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x > 80 else 0)

testanic['Cabin Aft?'] = testanic['Cabin_No'].apply(lambda x: 1 if x > 80 else 0)

In [24]:
traintanic['Cabin Amidships?'] = traintanic['Cabin_No'].apply(lambda x: 1 if x in range(66,81) else 0)

testanic['Cabin Amidships?'] = testanic['Cabin_No'].apply(lambda x: 1 if x in range(66,81) else 0)

In [25]:
traintanic['Sex-int'] = traintanic['Sex'].apply(lambda x: 1 if x == 'female' else 0)

testanic['Sex-int'] = testanic['Sex'].apply(lambda x: 1 if x == 'female' else 0)

In [60]:
traintanic['Woman or Child w/o Recorded Age?'] = traintanic.apply(lambda row: 1 if np.isnan(row['Age']) and \
                                re.compile('Master|Miss|Mrs.').search(row['Name']) else 0, axis=1)

testanic['Woman or Child w/o Recorded Age?'] = traintanic.apply(lambda row: 1 if np.isnan(row['Age']) and \
                                re.compile('Master|Miss|Mrs.').search(row['Name']) else 0, axis=1)

In [54]:
traintanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   PassengerId                      891 non-null    int64  
 1   Survived                         891 non-null    int64  
 2   Pclass                           891 non-null    int64  
 3   Name                             891 non-null    object 
 4   Sex                              891 non-null    object 
 5   Age                              714 non-null    float64
 6   SibSp                            891 non-null    int64  
 7   Parch                            891 non-null    int64  
 8   Ticket                           891 non-null    object 
 9   Fare                             891 non-null    float64
 10  Cabin                            204 non-null    object 
 11  Embarked                         889 non-null    object 
 12  Is Non-Anglo Western? 

In [56]:
traintanic.sample(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Cabin_No,Is Child?,Is Young Adult?,Is Not Young?,Big Family?,Cabin Fore?,Cabin Aft?,Cabin Amidships?,Sex-int,Woman or Child w/o Recorded Age
745,746,0,1,"Crosby, Capt. Edward Gifford",male,70.0,1,1,WE/P 5735,71.0000,...,22,0,0,1,0,1,0,0,0,0
56,57,1,2,"Rugg, Miss. Emily",female,21.0,0,0,C.A. 31026,10.5000,...,0,0,1,0,0,1,0,0,1,0
183,184,1,2,"Becker, Master. Richard F",male,1.0,2,1,230136,39.0000,...,4,1,0,0,0,1,0,0,0,0
90,91,0,3,"Christmann, Mr. Emil",male,29.0,0,0,343276,8.0500,...,0,0,1,0,0,1,0,0,0,0
203,204,0,3,"Youseff, Mr. Gerious",male,45.5,0,0,2628,7.2250,...,0,0,0,1,0,1,0,0,0,0
122,123,0,2,"Nasser, Mr. Nicholas",male,32.5,1,0,237736,30.0708,...,0,0,0,0,0,1,0,0,0,0
605,606,0,3,"Lindell, Mr. Edvard Bengtsson",male,36.0,1,0,349910,15.5500,...,0,0,0,0,0,1,0,0,0,0
645,646,1,1,"Harper, Mr. Henry Sleeper",male,48.0,1,0,PC 17572,76.7292,...,33,0,0,1,0,1,0,0,0,0
154,155,0,3,"Olsen, Mr. Ole Martin",male,NaN,0,0,Fa 265302,7.3125,...,0,0,0,0,0,1,0,0,0,0
474,475,0,3,"Strandberg, Miss. Ida Sofia",female,22.0,0,0,7553,9.8375,...,0,0,1,0,0,1,0,0,1,0


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [62]:
forest = RandomForestClassifier(n_estimators = 5000, random_state = 1)
traintanic_data, valitanic_data, traintanic_labels, valitanic_labels = train_test_split(traintanic[[\
'Is 1st Class?','Is 3rd Class?','Sex-int','SibSp','Parch',\
    'Is Non-Anglo Western?','Is Eastern?',\
        'Is Child?','Is Young Adult?','Is Not Young?','Woman or Child w/o Recorded Age?',\
            'Cabin Fore?','Cabin Aft?','Cabin Amidships?']],\
           traintanic['Survived'], test_size = 0.1)
forest.fit(traintanic_data, traintanic_labels)

print(forest.score(valitanic_data, valitanic_labels))

0.8444444444444444


In [63]:
testanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   PassengerId                       418 non-null    int64  
 1   Pclass                            418 non-null    int64  
 2   Name                              418 non-null    object 
 3   Sex                               418 non-null    object 
 4   Age                               332 non-null    float64
 5   SibSp                             418 non-null    int64  
 6   Parch                             418 non-null    int64  
 7   Ticket                            418 non-null    object 
 8   Fare                              417 non-null    float64
 9   Cabin                             91 non-null     object 
 10  Embarked                          418 non-null    object 
 11  Is Non-Anglo Western?             418 non-null    int64  
 12  Is Easte

In [64]:
testanic_data = testanic[[\
'Is 1st Class?','Is 3rd Class?','Sex-int','SibSp','Parch',\
    'Is Non-Anglo Western?','Is Eastern?',\
        'Is Child?','Is Young Adult?','Is Not Young?','Woman or Child w/o Recorded Age?',\
            'Cabin Fore?','Cabin Aft?','Cabin Amidships?']]

In [65]:
testanic_predict = forest.predict(testanic_data)

In [682]:
testanic['Survived'] = testanic_predict

In [32]:
feature_importances = list(forest.feature_importances_)
features_list = ['Is 1st Class?','Is 3rd Class?','Sex-int','SibSp','Parch',\
    'Is Non-Anglo Western?','Is Eastern?',\
        'Is Child?','Is Young Adult?','Is Not Young?',\
            'Cabin Fore?','Cabin Aft?','Cabin Amidships?']

In [33]:
for feature, value in zip(features_list, feature_importances):
    print(f'{feature}: {value}')

Is 1st Class?: 0.07792414268392207
Is 3rd Class?: 0.11067788279227217
Sex-int: 0.4023363996487142
SibSp: 0.11071014471840142
Parch: 0.08323594867559994
Is Non-Anglo Western?: 0.04606641884083667
Is Eastern?: 0.027352913623971045
Is Child?: 0.045173560766760136
Is Young Adult?: 0.03531662232448531
Is Not Young?: 0.030003945356536596
Cabin Fore?: 0.013732012914440891
Cabin Aft?: 0.010197449237047388
Cabin Amidships?: 0.007272558417012327
